In [1]:
!pip install datasets
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
!pip install -U lightrag[ollama]
!pip install rouge-score

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
######################################################################## 100.0%                                                    7.2%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [3]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [4]:
!ollama create myLLM -f /kaggle/input/modelfile-postfix/Modelfile

[GIN] 2024/12/18 - 17:47:39 | 200 |      30.274µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/12/18 - 17:47:39 | 200 |   24.438165ms |       127.0.0.1 | POST     "/api/create"
transferring model data 
using existing layer sha256:667b0c1932bc6ffc593ed1d03f895bf2dc8dc6df21db3042284a6f4416b06a29 
using existing layer sha256:948af2743fc78a328dcb3b0f5a31b3d75f415840fdb699e8b1235978392ecf85 
using existing layer sha256:0ba8f0e314b4264dfd19df045cde9d4c394a52474bf92ed6a3de22a4ca31a177 
creating new layer sha256:6b346e388b04ec0fac477285ecd957b14813d92703eb4c0cb3d02ba564036e69 
creating new layer sha256:50a0d6e597d12c01753e3f1530997e590ecafe25f2140bcb1879bdb180586c11 
writing manifest 
success 


In [5]:
!ollama list

[GIN] 2024/12/18 - 17:47:56 | 200 |      40.565µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/12/18 - 17:47:56 | 200 |     801.273µs |       127.0.0.1 | GET      "/api/tags"
NAME            ID              SIZE      MODIFIED       
myLLM:latest    c6c4178448c5    4.9 GB    16 seconds ago    
llama3.1:8b     46e0c10c039e    4.9 GB    17 seconds ago    


In [6]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import time


qa_template = r"""<SYS>
You are a helpful assistant. Just answer the question and avoid chatting, just give me the post-fix answer for the question. Just in case you don't find the answer or you are not sure give me this answer: "Not found!" .In case the question is not a numerical question this must be your output: "I only answer to numerical questions; this is I am a numerical assistant; this is irrelevent!"
Below is a set of examples from dataset, learn from them and answer according to the notation and pattern you see.
example1: 


</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})
        

In [7]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display



model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "myLLM"}
}
qa = SimpleQA(**model)
output=qa("what is happiness")
display(Markdown(f"**Answer:** {output.data}"))

time=2024-12-18T17:49:43.701Z level=INFO source=sched.go:714 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-667b0c1932bc6ffc593ed1d03f895bf2dc8dc6df21db3042284a6f4416b06a29 gpu=GPU-c8b76643-7a71-2f57-0660-b5f7ed27df64 parallel=1 available=16790978560 required="11.1 GiB"
time=2024-12-18T17:49:43.801Z level=INFO source=server.go:104 msg="system memory" total="31.4 GiB" free="30.0 GiB" free_swap="0 B"
time=2024-12-18T17:49:43.801Z level=INFO source=memory.go:356 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="11.1 GiB" memory.required.partial="11.1 GiB" memory.required.kv="4.0 GiB" memory.required.allocations="[11.1 GiB]" memory.weights.total="7.9 GiB" memory.weights.repeating="7.5 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="2.1 GiB" memory.graph.partial="2.2 GiB"
time=2024-12-18T17:49:43.80

[GIN] 2024/12/18 - 17:49:49 | 200 |   6.42253998s |       127.0.0.1 | POST     "/api/generate"


**Answer:** Happiness is a state of being content and fulfilled, often characterized by feelings of joy, satisfaction, and well-being. It can be achieved through various means such as positive relationships, good health, personal growth, and a sense of purpose.

In [8]:
prompt = """
other items on our consolidated financial statements have been appropriately adjusted from the amounts provided in the earnings release , including a reduction of our full year 2016 gross profit and income from operations by $ 2.9 million , and a reduction of net income by $ 1.7 million. .
( in thousands )	at december 31 , 2016	at december 31 , 2015	at december 31 , 2014	at december 31 , 2013	at december 31 , 2012
cash and cash equivalents	$ 250470	$ 129852	$ 593175	$ 347489	$ 341841
working capital ( 1 )	1279337	1019953	1127772	702181	651370
inventories	917491	783031	536714	469006	319286
total assets	3644331	2865970	2092428	1576369	1155052
total debt including current maturities	817388	666070	281546	151551	59858
total stockholders 2019 equity	$ 2030900	$ 1668222	$ 1350300	$ 1053354	$ 816922
( 1 ) working capital is defined as current assets minus current liabilities. Question: what was the percentage change in inventories from 2015 to 2016?

"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

time=2024-12-18T17:50:11.640Z level=WARN source=routes.go:273 msg="the context field is deprecated and will be removed in a future version of Ollama"


[GIN] 2024/12/18 - 17:50:12 | 200 |  956.767685ms |       127.0.0.1 | POST     "/api/generate"


**Answer:** Not found!

In [ ]:
prompt = """
convert from infix notation into postfix notation: 8 * (11+43) ÷ 67
"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
prompt = """
(12 * 67) ^ (23) - 152
"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
print("The true answer for above:")

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

generated = "Subtract(12.393, 22.366), divide(#0, 22.366)"
target = "subtract(22.366, 12.393), divide(#0, 12.393)"



scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(target, generated)

print("rouge1: ", scores['rouge1'])
print("rouge2: ", scores['rouge2'])
print("rougeL: ", scores['rougeL'])


# BLEU score calculation
bleu_score = sentence_bleu([target], generated)

print(f"BLEU score: {bleu_score}")

In [ ]:
from datasets import load_dataset

# Load FinQA dataset
dataset = load_dataset("ibm/finqa", trust_remote_code=True)

# Check the structure
print(dataset)
print(dataset['train'][0])  # View the first training example

In [ ]:

dataset['train'][10]['program_re']

In [ ]:
dataset['train'][10]['question']

In [ ]:
from transformers import AutoTokenizer

# Authentication token
token = "User Access Token for HuggingFace"

# Initialize tokenizer for Llama 3.1
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=token)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Preprocessing function with corrected target handling
def preprocess(example):
    # Safeguard for missing fields
    post_text = example.get('post_text', '')
    pre_text = example.get('pre_text', '')
    table = example.get('table', '')
    question = example.get('question', '')
    program = example.get('program_re', '')

    # Format input and target text
    input_text = f"post_text: {post_text} table: {table} pre_text: {pre_text} Question: {question}"
    target_text = program  

    # Tokenize input and target text
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=8192, return_tensors="pt")
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

    return {
        "input_ids": inputs["input_ids"][0],
        "attention_mask": inputs["attention_mask"][0],
        "labels_tokenized": targets["input_ids"][0]  # Tokenized target for training
    }


In [ ]:
processed_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names, batched=False)

# Set PyTorch format for DataLoader, but include `labels` as a string for ROUGE
processed_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels_tokenized"])


In [ ]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 16

# Create a DataLoader for batching
train_loader = DataLoader(processed_dataset["train"], batch_size=batch_size, shuffle=True)


scores = []

sample_limit = 128

count = 0
# Evaluation loop with ROUGE scoring
for batch in train_loader:
    count += 1
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels_tokenized = batch["labels_tokenized"]


    # Decode inputs to strings
    input_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]
    target_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_tokenized]

    
    # Generate model responses
    responses = []
    for input_text in input_texts:
        try:
            response = qa(input_text)
            responses.append(response.data)
        except Exception as e:
            responses.append("API_ERROR")
            print(f"API call failed: {e}")

    # Compute ROUGE scores
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    for target, response in zip(target_texts, responses): 
        scores.append(scorer.score(target, response))

    if sample_limit <= batch_size * count:
        break

In [ ]:
n = 10
input_str = input_texts[n]
response_text = responses[n] 
targ = target_texts[n]

print("input:", input_str)
print("\n\ntarget text:", targ)
print("\n\nresponce text:", response_text)

In [ ]:
print(len(input_texts))
print(len(target_texts))
print(len(responses))

In [ ]:
#n = 22
#n = 5
#n = 56
n = 9

input_str = input_texts[n]
response_text = responses[n] 
targ = target_texts[n]

print("input:", input_str)
print("\n\ntarget text:", targ)
print("\n\nresponce text:", response_text)

In [ ]:
for i in range(len(target_texts)):
    print(target_texts[i])

In [ ]:
for i in range(len(responses)):
    print(responses[i])

In [ ]:
suum = 0
for i in range(len(scores)):
    suum += float(scores[i]['rougeL'][2])
print(suum / len(scores))

In [ ]:
print(len(scores))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define two sentences
# sentence1 = "Subtract(12.393, 22.366), divide(#0, 22.366)"
# sentence2 = "subtract(22.366, 12.393), divide(#0, 12.393)"

sentence1 = "The capital of France is Paris."
sentence2 = "Paris is not the capital of France."


# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the sentences into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

# Calculate cosine similarity
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Print the similarity score
print(f"Cosine Similarity: {similarity[0][0]:.4f}")


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Correct answer and LLM output
correct_answer = "The capital of France is Paris."
llm_output = "Paris is not the capital of France."

# Generate embeddings
correct_embedding = model.encode(correct_answer, convert_to_tensor=True)
llm_embedding = model.encode(llm_output, convert_to_tensor=True)

# Compute cosine similarity
similarity = util.cos_sim(correct_embedding, llm_embedding)

print(f"Cosine Similarity: {similarity.item():.4f}")


In [ ]:
!pip install sentence_transformers